In [6]:
import os

In [7]:
import time

In [8]:
import pandas as pd
df = pd.read_csv("./data/100_verified_context_QA.csv")

In [ ]:
df

In [10]:
questions = []
ground_truths = []
answers = []
contexts = []

In [11]:
from langchain_cohere.chat_models import ChatCohere
llm = ChatCohere(
                    model="command-r-plus",
                    temperature=0.1,
                    max_tokens=None,
                    timeout=None,
                )

In [12]:
import ast
df['Answer'] = [ast.literal_eval(i) for i in df['Answers']]

In [13]:
df['Answer'] = [i[0] for i in df['Answer']]

In [ ]:
questions_pre = df['Question'].tolist()
ground_truths_pre = df['Answer'].tolist()

rag_class = rag.RAG()

# llm = rag.LLM(llm="command-r-plus-no-streaming").llm
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                   chain_type="stuff",
                                   retriever=rag_class.retriever,
                                   return_source_documents=True,
                                   chain_type_kwargs={
                                         "prompt": rag.Prompt(custom_prompt_testset).set_prompt(),
                                     },)  
stop_processing = False
for i in range(len(questions_pre)):
    if stop_processing:
        break
    retries = 0
    while retries < 3:
        try:
            print(str(i) + " " + questions_pre[i])
            result = qa_chain.invoke(questions_pre[i])
            questions.append(questions_pre[i])
            ground_truths.append(ground_truths_pre[i])
            answers.append(result['result'])
            contexts.append([x.page_content for x in result['source_documents']])
            break  # Exit the retry loop if successful
        except Exception as e:
            retries += 1
            if retries < 3:
                print(f"Error encountered: {e}. Retrying in 30 seconds... (Attempt {retries}/{3})")
                time.sleep(30)
            else:
                print(f"Error encountered: {e}. Max retries reached. Stopping the process.")
                stop_processing = True
                break

In [14]:
data = {
    "questions" : questions,
    "contexts" : contexts,
    "ground_truths" : ground_truths,
    "answers" : answers
}

In [15]:
import json
with open("./data/testset_10_samples.json", "w", encoding="utf-8") as outfile: 
    json.dump(data, outfile)

In [1]:
import pandas as pd
import ast
import time
import json
from langchain_cohere import ChatCohere
from langchain.chains import RetrievalQA
import rag
from rag_init import custom_prompt_testset
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
class CreateDataset:
    def __init__(self, csv_path, rag_class, custom_prompt_testset,cohere_api):
        self.df = pd.read_csv(csv_path)
        # self.df['Answer'] = [ast.literal_eval(i) for i in self.df['Answer']]
        # self.df['Answer'] = [i[0] for i in self.df['Answer']]
        # self.df = self.df.explode('Answer').reset_index(drop=True)
        self.questions_pre = self.df['Question'].tolist()
        self.ground_truths_pre = self.df['Answers'].tolist()
        self.cohere_api = cohere_api
        self.questions = []
        self.ground_truths = []
        self.answers = []
        self.contexts = []
        self.api_idx = 0
        # Initialize ChatCohere model
        # self.llm = ChatCohere(
        #     model="command-r-plus",
        #     temperature=0.1,
        #     max_tokens=None,
        #     timeout=None,
        #     cohere_api_key=self.cohere_api[self.api_idx]
        # )
        self.llm = ChatOpenAI(
            model="hf:Qwen/Qwen1.5-110B-Chat",    
            openai_api_key="glhf_7cacb0cf53d5065f1b46d6fcc98dd784",
            openai_api_base="https://glhf.chat/api/openai/v1",
        )

        
        # Initialize QA Chain with the RAG model and the custom prompt
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=rag_class.retriever,
            return_source_documents=True,
            chain_type_kwargs={
                "prompt": rag.Prompt(custom_prompt_testset).set_prompt(),
            },
        )
        
        self.stop_processing = False
    def change_api(self):
        self.llm = ChatOpenAI(
            model="hf:Qwen/Qwen1.5-110B-Chat",    
            openai_api_key="glhf_7cacb0cf53d5065f1b46d6fcc98dd784",
            openai_api_base="https://glhf.chat/api/openai/v1",
        )
        
        # Initialize QA Chain with the RAG model and the custom prompt
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=rag_class.retriever,
            return_source_documents=True,
            chain_type_kwargs={
                "prompt": rag.Prompt(custom_prompt_testset).set_prompt(),
            },
        )
    def process_data(self):
        # Loop through the questions and invoke the LLM
        for i in range(len(self.questions_pre)):
            if self.stop_processing:
                break
            retries = 0
            while retries < 4:
                try:
                    if retries == 3:
                        self.api_idx += 1
                        if self.api_idx == len(self.cohere_api):
                            self.api_idx = 0
                        self.change_api()
                        retries = 0
                    print(f"{i} {self.questions_pre[i]}")
                    result = self.qa_chain.invoke(self.questions_pre[i])
                    self.questions.append(self.questions_pre[i])
                    self.ground_truths.append(self.ground_truths_pre[i])
                    self.answers.append(result['result'])
                    print(result['result'])
                    self.contexts.append([x.page_content for x in result['source_documents']])
                    break
                except Exception as e:
                    retries += 1
                    if retries < 4:
                        print(f"Error encountered: {e}. Retrying in 30 seconds... (Attempt {retries}/{4})")
                        time.sleep(30)
                    else:
                        print(f"Error encountered: {e}. Max retries reached. Stopping the process.")
                        self.stop_processing = True
                        break
    
    def save_data(self, output_path):
        # Save the data to a JSON file
        data = {
            "questions": self.questions,
            "contexts": self.contexts,
            "ground_truths": self.ground_truths,
            "answers": self.answers
        }
        with open(output_path, "w", encoding="utf-8") as outfile:
            json.dump(data, outfile)
        print(f"Data saved to {output_path}")

d:\RAG4\.venv\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
USER_AGENT environment variable not set, consider setting it to identify your requests.


2024-10-07 09:23:03 - Loaded .env file
2024-10-07 09:23:19 - PyTorch version 2.4.1 available.
2024-10-07 09:23:20 - Use pytorch device_name: cpu
2024-10-07 09:23:20 - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


d:\RAG4\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


2024-10-07 09:27:46 - Use pytorch device: cpu


In [2]:
cohere_api = ['WOxxBY2ZfxTvxnDIVvCn0ibdR0maXHnARI3VbEKw','b5eKaSQ0zduU8djl66MDLbqNtReS3CWWH5ljxjtM']
dataset_creator = CreateDataset(csv_path="./data/100_400_processed_non_parahrased.csv", 
                                rag_class=rag.RAG(), 
                                custom_prompt_testset=custom_prompt_testset,cohere_api=cohere_api)
dataset_creator.process_data()
dataset_creator.save_data(output_path="./data/100_400_processed_non_parahrased_qwen.json")

0 Các hình thức nộp hồ sơ cho thủ tục Điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương là gì?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-07 09:27:57 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 200 OK"
Các hình thức nộp hồ sơ cho thủ tục Điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương bao gồm:
1. **Trực tiếp**: Nộp hồ sơ trực tiếp tại trụ sở Phòng Cảnh sát quản lý hành chính về trật tự xã hội, Công an tỉnh, thành phố trực thuộc Trung ương trong giờ hành chính các ngày làm việc.
2. **Trực tuyến**: Nộp hồ sơ qua Cổng Dịch vụ công quốc gia hoặc Cổng dịch vụ công Bộ Công an.
3. **Dịch vụ bưu chính**: Nộp hồ sơ qua dịch vụ bưu chính.
1 Cơ quan thực hiện và cơ quan có thẩm quyền cho thủ tục Điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương là gì?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-07 09:28:01 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 200 OK"
Cơ quan thực hiện thủ tục Điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ là Phòng Cảnh sát quản lý hành chính về trật tự xã hội tại Công an tỉnh, thành phố trực thuộc trung ương. Cơ quan có thẩm quyền cũng là Phòng Cảnh sát quản lý hành chính về trật tự xã hội.
2 Cơ quan, tổ chức, doanh nghiệp nào được phép vận chuyển tiền chất thuốc nổ?


...

2024-10-07 10:19:21 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-10-07 10:19:21 - Retrying request to /chat/completions in 0.964477 seconds
2024-10-07 10:19:24 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-10-07 10:19:24 - Retrying request to /chat/completions in 1.512702 seconds
2024-10-07 10:19:26 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Error encountered: Error code: 429 - {'error': 'Too many requests, try again later'}. Retrying in 30 seconds... (Attempt 1/4)
481 Cấp thực hiện của thủ tục "Đề nghị xác nhận tình trạng vũ khí, công cụ hỗ trợ đã mất tính năng, tác dụng được sử dụng để triển lãm, trưng bày hoặc làm đạo cụ trong hoạt động văn hóa, nghệ thuật" là gì?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-07 10:19:59 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-10-07 10:19:59 - Retrying request to /chat/completions in 0.840787 seconds
2024-10-07 10:20:01 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-10-07 10:20:01 - Retrying request to /chat/completions in 1.986715 seconds
2024-10-07 10:20:04 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Error encountered: Error code: 429 - {'error': 'Too many requests, try again later'}. Retrying in 30 seconds... (Attempt 2/4)
481 Cấp thực hiện của thủ tục "Đề nghị xác nhận tình trạng vũ khí, công cụ hỗ trợ đã mất tính năng, tác dụng được sử dụng để triển lãm, trưng bày hoặc làm đạo cụ trong hoạt động văn hóa, nghệ thuật" là gì?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-07 10:20:37 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-10-07 10:20:37 - Retrying request to /chat/completions in 0.978331 seconds
2024-10-07 10:20:38 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-10-07 10:20:38 - Retrying request to /chat/completions in 1.627853 seconds
2024-10-07 10:20:41 - HTTP Request: POST https://glhf.chat/api/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
Error encountered: Error code: 429 - {'error': 'Too many requests, try again later'}. Retrying in 30 seconds... (Attempt 3/4)
Error encountered: name 'rag_class' is not defined. Max retries reached. Stopping the process.
Data saved to ./data/100_400_processed_non_parahrased_qwen.json


In [7]:
data = {
    "questions": dataset_creator.questions[:100],
    "contexts": dataset_creator.contexts[:100],
    "ground_truths": dataset_creator.ground_truths[:100],
    "answers": dataset_creator.answers[:100]
}
with open("./data/50_200_0-100.json", "w", encoding="utf-8") as outfile:
    json.dump(data, outfile)

In [3]:
dataset_creator.answers

['Các hình thức nộp hồ sơ cho thủ tục Điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương là:\n\n- Nộp hồ sơ trực tiếp tại trụ sở Phòng Cảnh sát quản lý hành chính về trật tự xã hội, Công an tỉnh, thành phố trực thuộc Trung ương;\n- Nộp hồ sơ trực tuyến qua Cổng Dịch vụ công quốc gia hoặc Cổng dịch vụ công Bộ Công an;\n- Nộp hồ sơ qua dịch vụ bưu chính.',
 'Để nộp hồ sơ điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ tại Công an tỉnh, thành phố trực thuộc trung ương, cần thực hiện theo những phương thức sau:\n\n- Nộp hồ sơ trực tiếp tại trụ sở Phòng Cảnh sát quản lý hành chính về trật tự xã hội, Công an tỉnh, thành phố trực thuộc Trung ương;\n- Nộp hồ sơ trực tuyến qua Cổng Dịch vụ công quốc gia hoặc Cổng dịch vụ công Bộ Công an;\n- Nộp hồ sơ qua dịch vụ bưu chính.',
 'Khi muốn điều chỉnh Giấy phép vận chuyển tiền chất thuốc nổ, người dân cần nộp hồ sơ theo những cách thức sau tại Công an tỉnh, thành phố trực thuộc trung ương:\n\n- Nộ

In [1]:
import pandas as pd
df = pd.read_csv("./data/demotest.csv")

In [ ]:
df